<a href="https://colab.research.google.com/github/Rakeshkrishnamurthy/Rocky_Help/blob/main/Raw_Staging_TGT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pyspark.sql.window import Window

# Step 1: Initialize Spark session
spark = SparkSession.builder.appName("DataProcessingAndTransformation").getOrCreate()

# Step 2: Load raw data and handle null values and duplicates
raw_data_path = "raw/path"
raw_df = spark.read.parquet(raw_data_path)

# Handling null values
processed_df = raw_df.na.drop()

# Handling duplicates
processed_df = processed_df.dropDuplicates()

# Step 3: Load data to staging and perform validation
staging_path = "staging/path/managed"
processed_df.write.parquet(staging_path, mode="overwrite")

# Perform data and schema validation on staging data
# (Implement your validation logic here)

# Step 4: Load data to target and perform transformations
target_path = "target/path/extr"
transformed_df = spark.read.parquet(staging_path)

# a) Partition based on year and month
transformed_df = transformed_df.withColumn("year_month", col("date_column").substr(1, 7))

# b) Split the data in col_info column after 'expo_' and create a new column
transformed_df = transformed_df.withColumn("new_column", when(col("col_info").contains("expo_"), col("col_info").substr(6)).otherwise(None))

# c) Self join for credit payback information
window_spec = Window.partitionBy("client_id").orderBy("payment_date")
transformed_df = transformed_df.withColumn("previous_payment_date", lag("payment_date").over(window_spec))

# Additional transformations...
# (Implement your specific transformations here)

# Step 5: Save the output in Parquet format to production location
output_path = "target/path/prod"
transformed_df.write.parquet(output_path, mode="overwrite")

# Stop the Spark session
spark.stop()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lag
from pyspark.sql.window import Window

# Step 1: Initialize Spark session
spark = SparkSession.builder.appName("IncrementalDataProcessing").getOrCreate()

# Step 2: Load raw data and handle null values and duplicates
raw_data_path = "raw/path"
raw_df = spark.read.parquet(raw_data_path)

# Handling null values
processed_df = raw_df.na.drop()

# Handling duplicates
processed_df = processed_df.dropDuplicates()

# Step 3: Load data to staging and perform validation
staging_path = "staging/path/managed"
processed_df.write.mode("append").parquet(staging_path)

# Perform data and schema validation on staging data
# (Implement your validation logic here)

# Step 4: Load data to target and perform transformations
target_path = "target/path/extr"
existing_data = spark.read.parquet(target_path)

# Identify new or updated records
processed_df = processed_df.join(existing_data, ["common_key_column"], "leftanti")

# a) Partition based on year and month
processed_df = processed_df.withColumn("year_month", col("date_column").substr(1, 7))

# b) Split the data in col_info column after 'expo_' and create a new column
processed_df = processed_df.withColumn("new_column", when(col("col_info").contains("expo_"), col("col_info").substr(6)).otherwise(None))

# c) Self join for credit payback information
window_spec = Window.partitionBy("client_id").orderBy("payment_date")
processed_df = processed_df.withColumn("previous_payment_date", lag("payment_date").over(window_spec))

# Additional transformations...
# (Implement your specific transformations here)

# Step 5: Save the output in Parquet format to production location
processed_df.write.mode("append").parquet(target_path)

# Stop the Spark session
spark.stop()
